In [6]:
!pip install langchain-community --quiet

In [7]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="ENV/env.txt")


True

In [8]:
import requests, pprint

#URL complète avec la clé API et les coordonnées ici la clé API n'est pas sensible (gratuite, et de toute façon on travaille en GET)
url = "http://api.weatherapi.com/v1/current.json?key=98adae444b4947a3a4e91950250905&q=43.937,4.813&aqi=no"

# Requête GET
response = requests.get(url)

# Vérification
if response.status_code == 200:
    weather_json = response.json()
    print("✅ Données récupérées avec succès")
else:
    print(f"❌ Erreur {response.status_code} lors de la requête")

pprint.pprint(weather_json)

def format_meteo_phrase(weather_json):
    # Extraction des données
    cloud = weather_json["current"]["cloud"]
    precip = weather_json["current"]["precip_mm"]
    temp = weather_json["current"]["temp_c"]
    uv = weather_json["current"]["uv"]
    wind = weather_json["current"]["wind_kph"]

    # Construction de la phrase
    phrase = (
        f"Couverture nuageuse : {cloud}%, "
        f"Pluies : {precip} mm, "
        f"Température : {temp} degrés Celsius, "
        f"Indice UV : {uv}, "
        f"Vent : {wind} km/h"
    )

    return phrase

# Génération de la phrase météo
meteo_phrase = format_meteo_phrase(weather_json)
print(meteo_phrase)

✅ Données récupérées avec succès
{'current': {'cloud': 0,
             'precip_mm': 0.0,
             'temp_c': 27.2,
             'uv': 5.2,
             'wind_kph': 11.9},
 'location': {'country': 'France',
              'lat': 43.95,
              'localtime': '2025-08-05 11:06',
              'localtime_epoch': 1754384789,
              'lon': 4.817,
              'name': 'Avignon',
              'region': "Provence-Alpes-Cote d'Azur",
              'tz_id': 'Europe/Paris'}}
Couverture nuageuse : 0%, Pluies : 0.0 mm, Température : 27.2 degrés Celsius, Indice UV : 5.2, Vent : 11.9 km/h


In [12]:
# Extraction des variables
location = weather_json["location"]["name"]
temp = weather_json["current"]["temp_c"]
wind = weather_json["current"]["wind_kph"]
precip = weather_json["current"]["precip_mm"]
cloud = weather_json["current"]["cloud"]
uv = weather_json["current"]["uv"]
time = weather_json["location"]["localtime"]

In [50]:
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Rédige un bulletin pour Lieu : {location}, Date et heure : {time} avec {temp}°C, vent {wind} km/h, précipitations {precip} mm, couverture nuageuse {cloud} %, indice UV : {uv} UV. Important : commenter seulement le temps observé mais ne parles pas de ce qui est prévu. Ne précises pas si des précipitations sont attendues. Ajoute un conseil si nécessaire.")
llm = ChatOpenAI(model="gpt-3.5-turbo")
chain = LLMChain(llm=llm, prompt=prompt)

variables = {
    "location": location,
    "time": time,
    "temp": temp,
    "wind": wind,
    "precip": precip,
    "cloud": cloud,
    "uv": uv
}

result = chain.run(**variables)



In [51]:
print(result)

Bulletin météo pour Avignon, le 5 août 2025 à 11h06 :

Actuellement, il fait 27.2°C à Avignon avec un vent soufflant à 11.9 km/h. Le ciel est dégagé avec une couverture nuageuse de 0 %. L'indice UV est de 5.2 UV.

Conseil : Avec un indice UV élevé, il est recommandé de bien se protéger du soleil en utilisant de la crème solaire et en portant un chapeau.
